In [126]:
import random
from lang import load_data_int_seq
from utils import eq_encoder, is_eq_valid
from typing import List
from models.rl.env import IntegerSequenceEnv, get_current_position, encode_with_lang, decode_with_lang, eq_to_seq
import gym, ray
from ray.rllib.agents import ppo
import numpy as np

%load_ext autoreload
%autoreload 2

MAX_PENALTY_MAGNITUDE = 99.0

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [127]:
ray.init(ignore_reinit_error=True)


2021-07-07 16:30:39,680	INFO worker.py:745 -- Calling ray.init() again after it has already been called.


In [128]:
output_lang, input_lang, train, X_test, y_test = load_data_int_seq()

In [129]:
input_lang.n_words

4123

In [138]:
def compare_sequences(output_sequence: List[int], target_sequence: List[int]) -> float:
    # print("output_sequence ", output_sequence)
    # print("target_sequence ", output_sequence)
    magnitude: float = 0.0
    for x, y in zip(output_sequence, target_sequence):
        magnitude += abs(x - y)
    # magnitude /= len(norm_target_seq)
    return 10 - magnitude


In [139]:
def evaluate_candidate_eq(candidate: str, int_seq: List[int]) -> float:
    if is_eq_valid(candidate) == False:
        return -MAX_PENALTY_MAGNITUDE

    output_sequence = eq_to_seq(candidate, 9)

    if np.count_nonzero(output_sequence) < 1:
        return -MAX_PENALTY_MAGNITUDE

    return compare_sequences(output_sequence, int_seq)

In [140]:
env = IntegerSequenceEnv({"int_sequence": train[0][0], "output_length": 9, "input_lang": input_lang, "output_lang": output_lang, "evaluate": evaluate_candidate_eq})

In [141]:
for _ in range(0, 9):
    env.step(1)

In [142]:
env.step(1)

(([3, 3, 3, 3, 3, 3, 3, 3, 3], [2, 3, 4, 5, 6, 7, 8, 9]), -99.0, True, {})

In [143]:
env.reset()

([-1, -1, -1, -1, -1, -1, -1, -1, -1], [2, 3, 4, 5, 6, 7, 8, 9])

In [144]:
trainer = ppo.PPOTrainer(env=IntegerSequenceEnv, config={
    "env_config": {
        "int_sequence": train[0][0],
        "output_length": 9,
        "input_lang": input_lang,
        "output_lang": output_lang,
        "evaluate": evaluate_candidate_eq
    },
})

2021-07-07 16:32:52,268	INFO trainable.py:101 -- Trainable.setup took 12.112 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2021-07-07 16:32:52,270	WARNING util.py:53 -- Install gputil for GPU system monitoring.


In [145]:
trainer.train()

2021-07-07 16:32:59,111	WARNING ppo.py:237 -- The magnitude of your environment rewards are more than 17727.0x the scale of `vf_clip_param`. This means that it will take more than 17727.0 iterations for your value function to converge. If this is not intended, consider increasing `vf_clip_param`.


{'episode_reward_max': -62.0,
 'episode_reward_min': -76372038.0,
 'episode_reward_mean': -177266.36036036036,
 'episode_len_mean': 9.0,
 'episode_media': {},
 'episodes_this_iter': 444,
 'policy_reward_min': {},
 'policy_reward_max': {},
 'policy_reward_mean': {},
 'custom_metrics': {},
 'hist_stats': {'episode_reward': [-99.0,
   -99.0,
   -99.0,
   -99.0,
   -99.0,
   -99.0,
   -99.0,
   -99.0,
   -99.0,
   -99.0,
   -99.0,
   -99.0,
   -99.0,
   -99.0,
   -99.0,
   -99.0,
   -99.0,
   -99.0,
   -99.0,
   -99.0,
   -99.0,
   -99.0,
   -99.0,
   -99.0,
   -99.0,
   -99.0,
   -99.0,
   -99.0,
   -99.0,
   -99.0,
   -99.0,
   -99.0,
   -99.0,
   -99.0,
   -99.0,
   -99.0,
   -99.0,
   -99.0,
   -99.0,
   -99.0,
   -99.0,
   -99.0,
   -99.0,
   -99.0,
   -1534.0,
   -99.0,
   -99.0,
   -99.0,
   -99.0,
   -99.0,
   -99.0,
   -99.0,
   -99.0,
   -99.0,
   -99.0,
   -99.0,
   -99.0,
   -99.0,
   -99.0,
   -99.0,
   -99.0,
   -99.0,
   -99.0,
   -99.0,
   -99.0,
   -99.0,
   -99.0,
   -99.